In [1]:
import julia
julia.install()

In [5]:
from pathlib import Path

# %load_ext julia.magic
# %julia using Pkg
# from julia import Main
# from julia import Pkg

# # Point to the top of the project relative to this script
def projectdir(*args):
    return str(Path.cwd().joinpath("..", "..", "..", *args).resolve())

# # Pkg.add("AdaptiveResonance")
# Pkg.activate(str(projectdir()))

# from julia import AdaptiveResonance as ART
# art = ART.DDVFA()

In [2]:
from sklearn import datasets
from sklearn.model_selection import train_test_split

digits = datasets.load_digits()
# Take the first 500 data points: it's hard to see 1500 points
X = digits.data[:500]
y = digits.target[:500]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [3]:
X_train.transpose().shape
X_test.transpose().shape

(64, 165)

In [8]:
# from julia import Julia
# j = Julia()
from julia import Main as j

j.project_dir = projectdir()
j.eval("using Pkg; Pkg.activate(project_dir)")
j.eval("using AdaptiveResonance")
j.eval("art = DDVFA(rho_lb=0.4, rho_ub=0.75)")
print("Loaded DDVFA")

Loaded DDVFA


In [61]:
# ART.train!(art, X, y=y)
j.X_train = X_train.transpose()
j.y_train = y_train
j.X_test = X_test.transpose()
# j.eval("X_train_local = Matrix{Float64}(X_train)")
# j.eval("y_train_local = Vector{Int32}(y_train)")
j.eval("art.config = DataConfig(0.0, 16.0, 64)")
j.eval("train!(art, X_train, y=y_train)")
y_hat = j.eval("classify(art, X_test)")
# y_hat

In [62]:
n_weights = j.eval("AdaptiveResonance.get_n_weights(art)")

print(n_weights, X_train.shape)

165 (335, 64)


In [63]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_hat)

0.8242424242424242

In [26]:
from torch.utils.data import DataLoader
from julia import Main as j
from sklearn.metrics import accuracy_score

class DDVFAStrategy():
    """DDVFA Strategy"""

    # def __init__(self, model, optimizer, criterion):
    def __init__(self, criterion):
        # self.model = j.DDVFA()
        j.project_dir = projectdir()
        j.eval("using Pkg; Pkg.activate(project_dir)")
        j.eval("using AdaptiveResonance")
        j.eval("art = DDVFA(rho_lb=0.4, rho_ub=0.75)")
        j.eval("art.config = DataConfig(0.0, 255.0, 28*28)")

    def train(self, experience):
        # here you can implement your own training loop for each experience (i.e. 
        # batch or task).

        train_dataset = experience.dataset
        t = experience.task_label
        train_data_loader = DataLoader(
            train_dataset, num_workers=4, batch_size=128
        )

        for mb in train_data_loader:
            for sample in mb:
                j.sample = sample
                j.label = t
                j.eval("train!(art, sample, y=t)")

    def eval(self, experience):
        # here you can implement your own eval loop for each experience (i.e. 
        # batch or task).

        eval_dataset = experience.dataset
        t = experience.task_label
        eval_data_loader = DataLoader(
            eval_dataset, num_workers=4, batch_size=128
        )

        # for mb in eval_data_loader:
        #     for sample in mb:
        #         j.sample = sample
        #         y_hat = j.eval("classify(art, sample)")
        j.samples = mb
        y_hats = j.eval("classify(art, samples)")
        accuracy_score(y_test, y_hat)

        # eval here

In [ ]:
from avalanche.models import SimpleMLP
from avalanche.benchmarks import SplitMNIST

# Benchmark creation
benchmark = SplitMNIST(n_experiences=5)

# Create the Strategy Instance
cl_strategy = DDVFAStrategy()

# Training Loop
print('Starting experiment...')

for exp_id, experience in enumerate(benchmark.train_stream):
    print("Start of experience ", experience.current_experience)

    cl_strategy.train(experience)
    print('Training completed')

    print('Computing accuracy on the current test set')
    cl_strategy.eval(benchmark.test_stream[exp_id])

In [ ]:
"""
This example trains on Split CIFAR10 with Naive strategy.
In this example each experience has a different task label.
We use a multi-head model with a separate classifier for each task.
"""

import argparse
import torch
from torch.nn import CrossEntropyLoss
from torch.optim import Adam

from avalanche.benchmarks.classic import SplitCIFAR10
from avalanche.models import SimpleMLP, as_multitask
from avalanche.training.supervised import Naive


def main(args):

    # model
    model = SimpleMLP(input_size=32 * 32 * 3, num_classes=10)
    model = as_multitask(model, "classifier")

    # CL Benchmark Creation
    benchmark = SplitCIFAR10(n_experiences=5, return_task_id=True)
    train_stream = benchmark.train_stream
    test_stream = benchmark.test_stream

    # Prepare for training & testing
    optimizer = Adam(model.parameters(), lr=0.01)
    criterion = CrossEntropyLoss()

    # Choose a CL strategy
    strategy = Naive(
        model=model,
        optimizer=optimizer,
        criterion=criterion,
        train_mb_size=128,
        train_epochs=3,
        eval_mb_size=128,
        device=device,
    )

    # train and test loop
    for train_task in train_stream:
        strategy.train(train_task, num_workers=0)
        strategy.eval(test_stream)


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--cuda",
        type=int,
        default=0,
        help="Select zero-indexed cuda device. -1 to use CPU.",
    )
    args = parser.parse_args()
    main(args)


In [10]:
from avalanche.benchmarks import SplitMNIST
# from avalanche.benchmarks.utils.data_loader import GroupBalancedDataLoader
from avalanche.benchmarks.utils.data_loader import TaskBalancedDataLoader
benchmark = SplitMNIST(5, return_task_id=True)

dl = TaskBalancedDataLoader([exp.dataset for exp in benchmark.train_stream], batch_size=4)
for x, y, t in dl:
    print(t.tolist())
    break

AttributeError: 'list' object has no attribute 'targets_task_labels'